In [1]:
categorization_path = r"%USERPROFILE%\AppData\Local\activitywatch\activitywatch\aw-server\settings.json"
# instantiate the path
import os
categorization_path = categorization_path.replace('%USERPROFILE%', os.environ['USERPROFILE'])
print(categorization_path)

C:\Users\test\AppData\Local\activitywatch\activitywatch\aw-server\settings.json


In [2]:
import json

with open(categorization_path, 'r') as file:
    classes = json.load(file)['classes']

# print(classes)
# classes = classes[:5]
# print(classes)

classes = list(map(lambda x: (x["name"], x["rule"]), classes))
print(classes[:5])

# Extract a list of (name:List[str], rule:Dict)
# 
        # {
        #     "data": {
        #         "color": "#BFEF90"
        #     },
        #     "id": 8,
        #     "name": [
        #         "Explore",
        #         "PC"
        #     ],
        #     "rule": {
        #         "ignore_case": true,
        #         "regex": "\u6ce8\u518c\u8868|regedit|Taskmgr|Program Manager",
        #         "type": "regex"
        #     }
        # },

[(['Work'], {'type': 'none'}), (['Work', 'Programming'], {'ignore_case': True, 'regex': 'github|csdn|ipynb|python|powershell|terminal|vscode|cursor|GitHub|Stack Overflow|BitBucket|Gitlab|vim|Spyder|kate|Ghidra|Scite', 'type': 'regex'}), (['Work', 'PaperWork'], {'type': 'none'}), (['Work', 'Research'], {'ignore_case': True, 'regex': 'research|scholar|review|arxiv|zotero|overleaf|latex|paper|科研|论文|研究|NIPS|ICML|ICRA|AAMAS|CVPR|Conference', 'type': 'regex'}), (['Explore'], {'type': 'none'})]


In [45]:
import aw_client 
from aw_client import queries
import socket
from datetime import datetime, time, timedelta
from pprint import pprint
td1d = timedelta(days=1)
day_offset = timedelta(hours=4)

hostname = socket.gethostname()
now = datetime.now().astimezone()
today = (datetime.combine(now.date(), time()) + day_offset).astimezone()
print(today)

timeperiods = [(now - timedelta(minutes=5), now)]
timeperiods.reverse()


aw = aw_client.ActivityWatchClient()


param = queries.DesktopQueryParams(
        bid_window=f"aw-watcher-window_{hostname}",
        bid_afk=f"aw-watcher-afk_{hostname}",
        bid_browsers=[f"aw-watcher-web-edge",],
        classes=classes,
        include_audible=True,
    )

canonicalQuery = queries.canonicalEvents(
    param
)

# print(canonicalQuery.split(';'))
reseq_query = list(map(lambda x:x.replace(' ',''),canonicalQuery.split(';')))
print(reseq_query[-4])
reseq_query.insert(-3,
                   "events=union_no_overlap(browser_events,events)")

canonicalQuery = ";\n".join(reseq_query)

query = f"""
{canonicalQuery}
duration = sum_durations(events);
RETURN = {{"events": events, "browser_events":browser_events,"not_afk":not_afk, "duration": duration}};
"""

print(query)
# pprint(query)


res = aw.query(query, timeperiods)


2024-09-29 04:00:00+08:00

not_afk=period_union(not_afk,audible_events)

events=flood(query_bucket(find_bucket("aw-watcher-window_LAPTOP-E2E2PIVU")));



not_afk=flood(query_bucket(find_bucket("aw-watcher-afk_LAPTOP-E2E2PIVU")));

not_afk=filter_keyvals(not_afk,"status",["not-afk"]);


browser_events=[];

events_edge=flood(query_bucket("aw-watcher-web-edge"));

window_edge=filter_keyvals(events,"app",["msedge.exe","MicrosoftEdge"]);

events_edge=filter_period_intersect(events_edge,window_edge);

events_edge=split_url_events(events_edge);

browser_events=concat(browser_events,events_edge);

browser_events=sort_by_timestamp(browser_events);


audible_events=filter_keyvals(browser_events,"audible",[true]);

not_afk=period_union(not_afk,audible_events);
events=union_no_overlap(browser_events,events);


events=filter_period_intersect(events,not_afk);

events=categorize(events,[[["Work"],{"type":"none"}],[["Work","Programming"],{"ignore_case":true,"regex":"github|csdn|ipynb|python|powershell

In [46]:

pprint (res[0]['events'])


# pprint (res[0]['not_afk'])

# pprint(res[0]['browser_events'])

[{'data': {'$category': ['Work', 'Programming'],
           'app': 'Cursor.exe',
           'title': 'get_ratio.ipynb - aw-pop - Cursor [Administrator]'},
  'duration': 4.62,
  'id': 449326,
  'timestamp': '2024-09-29T15:18:27.107000+00:00'},
 {'data': {'$category': ['Work', 'Programming'],
           'app': 'Cursor.exe',
           'title': '● get_ratio.ipynb - aw-pop - Cursor [Administrator]'},
  'duration': 8.277,
  'id': 449327,
  'timestamp': '2024-09-29T15:18:31.727000+00:00'},
 {'data': {'$category': ['Work', 'Programming'],
           'app': 'Cursor.exe',
           'title': 'get_ratio.ipynb - aw-pop - Cursor [Administrator]'},
  'duration': 12.34,
  'id': 449328,
  'timestamp': '2024-09-29T15:18:40.004000+00:00'},
 {'data': {'$category': ['Work', 'Programming'],
           'app': 'Cursor.exe',
           'title': '● get_ratio.ipynb - aw-pop - Cursor [Administrator]'},
  'duration': 1.028,
  'id': 449329,
  'timestamp': '2024-09-29T15:18:52.344000+00:00'},
 {'data': {'$category

: 

In [5]:


from toolz.curried import reduceby
from aw_core import Event
from aw_transform import flood
from typing import List


smallevent = (res[0]['events'])
smallevent = flood(map(lambda x: Event(**x),smallevent), 0) ##type:ignore

cats= reduceby(lambda x: x['data']['$category'].__repr__(), 
            lambda acc,x: acc+x['duration'].total_seconds(), 
            smallevent,
            0
            )

print(cats)


Gap was of negative duration but could be safely merged (-67.296s). This message will only show once per batch.


{"['Uncategorized']": 15086.216999999995, "['Work', 'Research']": 702.758, "['Work', 'Programming']": 18289.15200000002}


In [5]:

dur = res[0]['duration']
print(res[0]['duration'])
print(cats.values())
# print(sum(map(lambda x: x.total_seconds(),cats.values())))

catratio = {k:v/dur for k,v in cats.items()}
print(catratio)

18684.778
dict_values([7581.233000000004, 700.321, 10403.224000000011])
{"['Uncategorized']": 0.40574380921196945, "['Work', 'Research']": 0.03748083065263072, "['Work', 'Programming']": 0.5567753601354007}


In [6]:
from treetype import TreeType
    
tree = TreeType.tree_expand(catratio)
print((tree['Work']).sum())
print(tree)

0.5942561907880315
{'Uncategorized': {'__root__': 0.40574380921196945}, 'Work': {'Research': {'__root__': 0.03748083065263072}, 'Programming': {'__root__': 0.5567753601354007}}}


In [15]:
cons =[
        {
            "term": ["Work"],
            "op": "ge",
            "value": 0.5
        },
        {
            "term": ["Work"],
            "op": "ge",
            "value": 0.6
        }
    ]
# from functools import reduce
from toolz.curried import curry
from fn import F
from toolz.curried import reduce,juxt
import operator

get_term = (F()
            << TreeType.sum
            << reduce(lambda acc,y:acc[y])
            )

print(get_term(['Work'],tree))

# res = ops(get_term())
def check_a_constraint(constraint,tree):
    return getattr(operator,constraint['op']) (tree.get_term(constraint['term']).sum(),constraint['value'])
    

print(all(juxt(map(curry(check_a_constraint),cons))(tree)))


0.5942561907880315
False
